<a href="https://colab.research.google.com/github/ProfDoof/advent_of_code/blob/2022/day10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 

## Load Data

In [21]:
from pathlib import Path

DAY = 10
DATA_FILE = Path.cwd() / 'drive' / 'MyDrive' / 'AdventOfCode' / 'aoc_data' / f'day{DAY}.txt'

data = DATA_FILE.read_text()

## Solution

### Test Values

In [22]:
larger_test = '''
addx 15
addx -11
addx 6
addx -3
addx 5
addx -1
addx -8
addx 13
addx 4
noop
addx -1
addx 5
addx -1
addx 5
addx -1
addx 5
addx -1
addx 5
addx -1
addx -35
addx 1
addx 24
addx -19
addx 1
addx 16
addx -11
noop
noop
addx 21
addx -15
noop
noop
addx -3
addx 9
addx 1
addx -3
addx 8
addx 1
addx 5
noop
noop
noop
noop
noop
addx -36
noop
addx 1
addx 7
noop
noop
noop
addx 2
addx 6
noop
noop
noop
noop
noop
addx 1
noop
noop
addx 7
addx 1
noop
addx -13
addx 13
addx 7
noop
addx 1
addx -33
noop
noop
noop
addx 2
noop
noop
noop
addx 8
noop
addx -1
addx 2
addx 1
noop
addx 17
addx -9
addx 1
addx 1
addx -3
addx 11
noop
noop
addx 1
noop
addx 1
noop
noop
addx -13
addx -19
addx 1
addx 3
addx 26
addx -30
addx 12
addx -1
addx 3
addx 1
noop
noop
noop
addx -9
addx 18
addx 1
addx 2
noop
noop
addx 9
noop
noop
noop
addx -1
addx 2
addx -37
addx 1
addx 3
noop
addx 15
addx -21
addx 22
addx -6
addx 1
noop
addx 2
addx 1
noop
addx -10
noop
noop
addx 20
addx 1
addx 2
addx 2
addx -6
addx -11
noop
noop
noop
'''

In [23]:
test = '''
noop
addx 3
addx -5
'''

### Solution Code

In [24]:
from typing import List
from collections import deque

class Cpu:
    def __init__(self):
        self.inputs = deque()
        self.registers = {'X': 1}
        self.state = Waiting(self)

    def is_accepting_input(self):
        return self.state.accepts_input

    def input_(self, input_: str):
        self.inputs.append(input_)

    def input_many(self, inputs: List[str]):
        self.inputs.extend(inputs)
        
    def tick(self):
        if len(self.inputs) > 0 and self.state.accepts_input:
            self.state = self.state.input_(self.inputs.popleft())
        self.state = self.state.tick()

    def is_working(self):
        return self.state.is_working or len(self.inputs) > 0

    def __str__(self) -> str:
        return f'CPU ({self.state})\n---------------\n{self.registers}\n---------------\n{self.inputs}'
    

class State:
    def __init__(self, cpu: Cpu, name: str, accepts_input: bool, is_working: bool):
        self.cpu = cpu
        self.name = name
        self.accepts_input = accepts_input
        self.is_working = is_working

    def __str__(self) -> str:
        return f'{self.name}'

    def input_(self, input_: str):
        raise Exception('Not Implemented')

    def tick(self):
        raise Exception('Not Implemented')


class NoOp(State):
    def __init__(self, cpu: Cpu):
        super().__init__(cpu, 'NoOp', False, True)
    
    def tick(self):
        return Waiting(self.cpu)


class AddX(State):
    def __init__(self, cpu: Cpu, x: int):
        super().__init__(cpu, 'AddX', False, True)
        self.ticked = False
        self.x = x

    def tick(self):
        if not self.ticked:
            self.ticked = True
            return self
        else:
            self.cpu.registers['X'] += self.x
            return Waiting(self.cpu)


class Waiting(State):
    def __init__(self, cpu: Cpu):
        super().__init__(cpu, 'Waiting', True, False)

    def input_(self, input_: str):
        split_input = input_.split(' ')
        if split_input[0] == 'noop':
            return NoOp(self.cpu)
        elif split_input[0] == 'addx' and len(split_input) == 2:
            return AddX(self.cpu, int(split_input[1]))
        
        raise Exception('Unacceptable Input')

In [34]:
class Crt:
    def __init__(self, cpu: Cpu):
        self.cpu = cpu
        self.screen = [['.' for i in range(40)] for j in range(6)]
        self.row = 0
        self.col = 0

    def tick(self):
        middle = self.cpu.registers['X']
        if self.col in set(range(middle - 1, middle + 2)):
            self.screen[self.row][self.col] = '#'

        self.col += 1
        if self.col % 40 == 0:
            self.col = 0
            self.row = (self.row + 1) % 6


    def __str__(self) -> str:
        return '\n'.join((' '.join(row) for row in self.screen))

input_data = data
cpu = Cpu()
crt = Crt(cpu)
logged_values = []
cycle = 1

log_pred = lambda cycle: (cycle + 20) % 40 == 0
log_func = lambda cycle, register: cycle * register

cpu.input_many(list(input_data.strip().split('\n')))
while cpu.is_working():
    if log_pred(cycle):
        logged_values.append(log_func(cycle, cpu.registers['X']))   
    crt.tick()
    cpu.tick()
    # print(cpu)
    # print(crt)
    cycle += 1

print(sum(logged_values))
print(crt)

14860
# # # . . . # # . . # # # # . # # # # . # . . # . # . . # . # # # . . # . . # .
# . . # . # . . # . . . . # . # . . . . # . . # . # . . # . # . . # . # . # . .
# . . # . # . . . . . . # . . # # # . . # # # # . # . . # . # . . # . # # . . .
# # # . . # . # # . . # . . . # . . . . # . . # . # . . # . # # # . . # . # . .
# . # . . # . . # . # . . . . # . . . . # . . # . # . . # . # . # . . # . # . .
# . . # . . # # # . # # # # . # # # # . # . . # . . # # . . # . . # . # . . # .
